## Imports

In [0]:
from pyspark.sql.functions import col
from pyspark.sql import Window
import pyspark.sql.functions as F
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator


import random

import mlflow
print(mlflow.__version__)

import os
os.environ['PYSPARK_PIN_THREAD'] = 'false'
spark.conf.set("spark.databricks.mlflow.trackMLlib.enabled", "true")

RANDOM_SEED = 0
# Define experiment name with proper Databricks path
EXPERIMENT_NAME = "/Shared/team_2_2/mlflow-baseline"
# Create the experiment if it doesn't exist
try:
    experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
    if experiment is None:
        experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
        print(f"Created new experiment with ID: {experiment_id}")
    else:
        print(f"Using existing experiment: {experiment.name}")
    mlflow.set_experiment(EXPERIMENT_NAME)
except Exception as e:
    print(f"Error with experiment setup: {e}")
    # Fallback to default experiment in workspace
    mlflow.set_experiment(f"/Users/{dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()}/default")



## Helper Functions


In [0]:
def checkpoint_dataset(dataset, file_path):
    # Create base folder
    section = "2"
    number = "2"
    base_folder = f"dbfs:/student-groups/Group_{section}_{number}"
    dbutils.fs.mkdirs(base_folder)
    # Create subfolders if file_path contains directories
    full_path = f"{base_folder}/{file_path}.parquet"
    subfolder = "/".join(full_path.split("/")[:-1])
    dbutils.fs.mkdirs(subfolder)
    # Save dataset as a parquet file
    dataset.write.mode("overwrite").parquet(full_path)
    print(f"Checkpointed {file_path}")

## Datasets

In [0]:
# Read in feature engineered custom joined data
train_df = spark.read.parquet("dbfs:/student-groups/Group_2_2/3_month_custom_joined/feature_eng/train.parquet")
validation_df = spark.read.parquet("dbfs:/student-groups/Group_2_2/3_month_custom_joined/feature_eng/validation.parquet")

df = train_df.unionByName(validation_df)

df = df.filter(F.col("CANCELLED") != 1)
print(df.count())
display(df.limit(10))

In [0]:
# combine date and scheduled departure time

df = df.withColumn(
    "utc_timestamp",
    F.to_timestamp(
        F.concat(
            F.col("FL_DATE"),
            F.lit(" "),
            F.lpad(F.col("CRS_DEP_TIME").cast("string"), 4, "0")
        ),
        "yyyy-MM-dd HHmm"
    )
)

# Create Splits for CV

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Truncate timestamp to hour level
df_indexed = df.withColumn(
    "hour", 
    F.date_trunc("hour", F.col("utc_timestamp"))
)

# Create time index based on unique hours
window_spec = Window.orderBy("hour")
df_indexed = df_indexed.withColumn(
    "time_idx", 
    F.dense_rank().over(window_spec)
)

df_indexed.display()

# 3 M splits config

In [0]:
max_time_idx = df_indexed.agg(F.max("time_idx")).collect()[0][0]
print(f"  Max time index: {max_time_idx}")

train_size = 720      # 30 days (720 hours)
gap_size = 2          # 2 hours
val_size = 168        # 7 days (168 hours)
step_size = 90       # Calculated to get exactly 10 folds

fold_window_size = train_size + gap_size + val_size
n_folds = (max_time_idx - fold_window_size) // step_size + 1
print(f"Step 2: Calculated {n_folds} folds")

# 1 Year splits config

In [0]:
# max_time_idx = df_indexed.agg(F.max("time_idx")).collect()[0][0]
# print(f"  Max time index: {max_time_idx}")

# train_size = 720*4      # 30 days (720 hours)
# gap_size = 2          # 2 hours
# val_size = 168*4        # 7 days (168 hours)
# step_size = 130*4       # Calculated to get exactly 10 folds

# fold_window_size = train_size + gap_size + val_size
# n_folds = (max_time_idx - fold_window_size) // step_size + 1
# print(f"Step 2: Calculated {n_folds} folds")

In [0]:
fold_mapping = []

for fold_id in range(1, n_folds + 1):
    fold_start = 1 + (fold_id - 1) * step_size
    # print(fold_id, fold_start)
    for t in range(fold_start, fold_start + train_size):
        fold_mapping.append((t, fold_id, "train"))
    
    for t in range(fold_start + train_size, fold_start + train_size + gap_size):
        fold_mapping.append((t, fold_id, "gap"))

    for t in range(fold_start + train_size + gap_size, fold_start + train_size + gap_size + val_size):
        fold_mapping.append((t, fold_id, "validation"))

fold_df = spark.createDataFrame(fold_mapping, ["time_idx", "fold_id", "split_type"])

result = df_indexed.join(
    F.broadcast(fold_df),
    on='time_idx',
    how='inner'
    )


In [0]:
if input("Careful! About to overwrite splits. If you want to continue, type y") == "y":
    result.write \
    .partitionBy("fold_id", "split_type") \
    .mode("overwrite") \
    .parquet(f"dbfs:/student-groups/Group_2_2/3_month_custom_joined/cv_splits")

## How to read the CV splits

In [0]:
def read_specific_fold(path: str, fold_id: int, split_type: str):
    return spark.read.parquet(f"{path}/fold_id={fold_id}/split_type={split_type}")

fold_1_val = read_specific_fold(path=f"dbfs:/student-groups/Group_2_2/3_month_custom_joined/cv_splits", fold_id=1, split_type="validation")


In [0]:
# Categorical encoding
carrier_indexer = StringIndexer(inputCol="OP_CARRIER", outputCol="carrier_idx", handleInvalid="keep")
origin_indexer = StringIndexer(inputCol="ORIGIN_AIRPORT_SEQ_ID", outputCol="origin_idx", handleInvalid="keep")
dest_indexer = StringIndexer(inputCol="DEST_AIRPORT_SEQ_ID", outputCol="dest_idx", handleInvalid="keep")
tail_num_indexer = StringIndexer(inputCol="TAIL_NUM", outputCol="tail_num_idx", handleInvalid="keep")

carrier_encoder = OneHotEncoder(inputCol="carrier_idx", outputCol="carrier_vec")
origin_encoder = OneHotEncoder(inputCol="origin_idx", outputCol="origin_vec")
dest_encoder = OneHotEncoder(inputCol="dest_idx", outputCol="dest_vec")
tail_num_encoder = OneHotEncoder(inputCol="tail_num_idx", outputCol="tail_num_vec")


# Assemble all features
assembler = VectorAssembler(
    inputCols=[
        "QUARTER",
        "MONTH", 
        "YEAR",
        "DAY_OF_MONTH",
        "DAY_OF_WEEK",
        "carrier_vec",
        "origin_vec",
        "dest_vec",
        "tail_num_vec",
        "CRS_ELAPSED_TIME",
        "DISTANCE",
        'HourlyDryBulbTemperature',
        'HourlyDewPointTemperature',
        'HourlyRelativeHumidity',
        'HourlyAltimeterSetting',
        'HourlyVisibility',
        'HourlyStationPressure',
        'HourlyWetBulbTemperature',
        'HourlyPrecipitation',
        'HourlyCloudCoverage',
        'HourlyCloudElevation',
        'HourlyWindSpeed'  
    ],
    outputCol="features"
)

In [0]:
# linear regression baseline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from mlflow.models import infer_signature

mlflow.spark.autolog()
with mlflow.start_run(run_name="lr - weather baseline 3m"):
    MODEL_NAME = "LR_WEATHER_BASELINE_3M"

    linear_reg = LinearRegression(
        featuresCol="features",
        labelCol="DEP_DELAY_NEW",
        # Linear Regression has different parameters than Random Forest
        maxIter=10, 
        regParam=0.3
    )

    # rf = RandomForestRegressor(
    #     featuresCol="features",  
    #     labelCol="DEP_DELAY_NEW",   
    #     numTrees=20,
    #     maxDepth=10
    # )

    # Create pipeline
    pipeline = Pipeline(stages=[
        carrier_indexer, origin_indexer, dest_indexer, tail_num_indexer,
        carrier_encoder, origin_encoder, dest_encoder, tail_num_encoder,
        assembler,
        linear_reg
        # rf
    ])

    model = pipeline.fit(train_3m_df)
    training_predictions = model.transform(train_3m_df)
    validation_predictions = model.transform(validation_3m_df)

    mae_evaluator = RegressionEvaluator(
        labelCol="DEP_DELAY_NEW",      
        predictionCol="prediction", 
        metricName="mae"           
    )

    rmse_evaluator = RegressionEvaluator(
        labelCol="DEP_DELAY_NEW",      
        predictionCol="prediction", 
        metricName="rmse"
    )

    # Calculate MAE
    mae_t = mae_evaluator.evaluate(training_predictions)
    mae_v = mae_evaluator.evaluate(validation_predictions)
    # Calculate RMSE
    rmse_t = rmse_evaluator.evaluate(training_predictions)
    rmse_v = rmse_evaluator.evaluate(validation_predictions)

    signature = infer_signature(train_df, training_predictions)

    mlflow.spark.log_model(
        model, 
        MODEL_NAME,
        input_example=train_df.limit(1).toPandas(),
        signature=signature,
        registered_model_name="flight_delay_prediction_baseline"
        )

    mlflow.log_metric("train_mae", mae_t)
    mlflow.log_metric("validation_mae", mae_v)
    mlflow.log_metric("train_rmse", rmse_t)
    mlflow.log_metric("validation_rmse", rmse_v)

In [0]:
def train_cv_models(train_pipeline, evaluator, n_folds=10):
  cv_results = []
  cv_models = []
  
  for fold_id in range(1, n_folds + 1):
    fold_train = read_specific_fold(path=f"dbfs:/student-groups/Group_2_2/1_year_custom_joined/cv_splits", fold_id=fold_id, split_type="train")
    fold_1_val = read_specific_fold(path=f"dbfs:/student-groups/Group_2_2/1_year_custom_joined/cv_splits", fold_id=1, split_type="validation")
    
    print(f"Training fold {fold_id}...")
    train_pipeline.fit(fold_train)



